## Домашнее задание по теме: "Функции и работа с данными"

### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:  
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [3]:
# импортируем данные из файлов
movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

In [4]:
# проверяем датафреймы
print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [5]:
def classification(rating):
    '''Функция для классификации фильмов по рейтингу'''
    if rating <= 2:
        return 'Low'
    elif rating <= 4:
        return 'Medium'
    else:
        return 'High'

In [6]:
# создание датафрейма со средней пользовательской оценкой
mean_ratings = ratings.groupby('movieId').mean().reset_index()[['movieId', 'rating']]

In [7]:
# применение функции к датафрейму
mean_ratings['class'] = mean_ratings['rating'].apply(classification) 

In [8]:
# объединение с датафреймом movies
fulldf = movies.merge(mean_ratings)[['title', 'genres', 'rating', 'class']]

In [9]:
# проверка
fulldf.sample(10)

,title,genres,rating,class
6667,"Invisible, The (2007)",Crime|Drama|Fantasy|Mystery|Thriller,3.000000,Medium
4266,Real Women Have Curves (2002),Comedy|Drama,3.500000,Medium
8167,Vamps (2012),Comedy|Horror|Romance,1.500000,Low
7512,Cemetery Junction (2010),Comedy|Drama|Romance,3.333333,Medium
8118,"Hunt, The (Jagten) (2012)",Drama,4.250000,High
2758,Whatever It Takes (2000),Comedy|Romance,4.000000,Medium
9015,SOMM: Into the Bottle (2016),Documentary,2.500000,Medium
5935,Ong-Bak: The Thai Warrior (Ong Bak) (2003),Action|Thriller,3.250000,Medium
7309,"3 Worlds of Gulliver, The (1960)",Adventure|Fantasy,4.000000,Medium
8293,"Internship, The (2013)",Comedy,2.833333,Medium


### Задание 2  
Используем файл keywords.csv.  
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность  
определенному региону. Т.е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется  
название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.  
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:  
```
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [10]:
# импортируем данные из csv
keywords = pd.read_csv('./ml-latest-small/keywords.csv')
# проверяем, что данные загружены
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [14]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [15]:
def geo_classification(keywords, geo_data):
    '''Функция для классификации ключевых слов по регионам'''
    for keyword in keywords.split():
        for region, cities in geo_data.items():
            if keyword in cities:
                return region
    return 'undefined'

In [16]:
keywords['region'] = keywords['keyword'].apply(geo_classification, geo_data=geo_data)

In [60]:
# проверка
keywords[keywords['region'] == 'Центр'].head()

,keyword,shows,region
127,авито москва,979292,Центр
849,авито ярославль,209581,Центр
1259,москва,153050,Центр
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр


### Задание 3 (бонусное)

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010 года.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:

    1. для каждой строки пройдите по всем годам списка years;
  
    2. если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
  
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

In [2]:
movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')

In [3]:
years = list(range(1950, 2011))

In [4]:
import numpy as np
movies['year'] = np.nan

In [6]:
def production_year(years, _str):
    '''Функция для получения года выпуска фильма'''
    for year in years:
        if str(year) in _str:
            return year

    return 1900

In [7]:
movies['year'] = movies.apply(lambda row: production_year(years, row['title']), axis=1).to_list()


In [8]:
movies.sample(10)

,movieId,title,genres,year
928,1168,Bad Moon (1996),Action|Adventure|Horror,1996
3245,4048,Imaginary Crimes (1994),Drama,1994
1650,2090,"Rescuers, The (1977)",Adventure|Animation|Children|Crime|Drama,1977
1372,1744,Firestorm (1998),Action|Adventure|Thriller,1998
2110,2636,"Mummy's Ghost, The (1944)",Horror,1900
582,685,It's My Party (1996),Drama,1996
137,158,Casper (1995),Adventure|Children,1995
4277,5696,Urban Cowboy (1980),Drama,1980
8040,94024,Louis Theroux: The Most Hated Family in Americ...,Documentary,1900
3145,3936,Phantom of the Opera (1943),Horror|Musical|Thriller,1900


In [15]:
# объединение с датафреймом movies
fulldf = movies.merge(ratings)[['year', 'rating']]

In [29]:
fulldf = fulldf.groupby(['year']).mean().sort_values(by=['rating'], ascending=False).reset_index()

In [30]:
fulldf.head(25)

,year,rating
0,1957,4.014241
1,1972,4.011136
2,1952,4.000000
3,1954,3.994220
4,1951,3.983539
5,1974,3.978704
6,1962,3.952446
7,1950,3.915254
8,1977,3.905786
9,1964,3.841492
